In [1]:
# sc master - running locally
sc.master

'local[*]'

In [2]:
import warnings
import time # For measuring the code execution time

warnings.filterwarnings("ignore")

In [3]:
sc = spark.sparkContext

In [4]:
purchasesDirectoryPath = "./data/area-grocery-purchase/"
categoriesDirectoryPath = "/data/"
purchasesDirectoryPathWithFile = "file:///home/hduser/MScDA-Sem2-CA1/data/area-grocery-purchase/"

In [42]:
from pyspark.sql.functions import lit

def extractAggregate(filename:str):
    start = filename.find('_')+1
    stop = filename[start:].find('_')+start
    return filename[start:stop]

def extractAndCreateDataFrame(filename:str):
    month=filename[:3]
    aggregate = extractAggregate(filename)
    tempDF = spark.read.csv(purchasesDirectoryPathWithFile + filename, header=True)
    tempDF = tempDF.withColumn("month", lit(month))
    tempDF = tempDF.withColumn("aggregate", lit(aggregate))
    return tempDF

In [43]:
from functools import reduce
import os

# get all the files from the aera grocery purchase CSV
files = os.listdir(purchasesDirectoryPath)

start = time.time()
dataframes = []

for filename in files:
    dataframes.append(extractAndCreateDataFrame(filename))

purchaseDF = reduce(lambda x, y: x.union(y), dataframes)

end = time.time()

elapsedTime = end - start;

In [44]:
print("Elapsed time: ", elapsedTime)

Elapsed time:  6.792416572570801


In [45]:
purchaseDF.printSchema()

root
 |-- area_id: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- weight_perc2.5: string (nullable = true)
 |-- weight_perc25: string (nullable = true)
 |-- weight_perc50: string (nullable = true)
 |-- weight_perc75: string (nullable = true)
 |-- weight_perc97.5: string (nullable = true)
 |-- weight_std: string (nullable = true)
 |-- weight_ci95: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- volume_perc2.5: string (nullable = true)
 |-- volume_perc25: string (nullable = true)
 |-- volume_perc50: string (nullable = true)
 |-- volume_perc75: string (nullable = true)
 |-- volume_perc97.5: string (nullable = true)
 |-- volume_std: string (nullable = true)
 |-- volume_ci95: string (nullable = true)
 |-- fat: string (nullable = true)
 |-- fat_perc2.5: string (nullable = true)
 |-- fat_perc25: string (nullable = true)
 |-- fat_perc50: string (nullable = true)
 |-- fat_perc75: string (nullable = true)
 |-- fat_perc97.5: string (nullable = true)
 |-- f

In [34]:
purchaseDF.createOrReplaceTempView("purchases")

2023-10-05 15:56:47,309 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [37]:
sampleDF = spark.sql("SELECT _c0, month, aggregate FROM purchases LIMIT 5")
sampleDF.show()

+---------+-----+---------+
|      _c0|month|aggregate|
+---------+-----+---------+
|  area_id|  May|   osward|
|E05000026|  May|   osward|
|E05000027|  May|   osward|
|E05000028|  May|   osward|
|E05000029|  May|   osward|
+---------+-----+---------+

